# Introduction to ffsim

ffsim is a software library for simulating fermionic quantum circuits that conserve particle number and the Z component of spin. By taking advantage of these symmetries, it can simulate these circuits much more efficiently than a generic quantum circuit simulator.

The primary way of using ffsim is by calling functions that transform statevectors represented directly as NumPy arrays. As an example, the following code shows how to create a vector representing the Hartree-Fock state with 6 spatial orbitals, 3 alpha electrons and 2 beta electrons, and then apply an orbital rotation to it. It also shows an equivalent way to construct the resulting state as a Slater determinant.

In [1]:
import numpy as np
import ffsim

# Set the number of orbitals and their occupancies
norb = 6
nelec = (3, 2)

# Create the Hartree-Fock state
vec = ffsim.hartree_fock_state(norb, nelec)

# Generate a random orbital rotation
orbital_rotation = ffsim.random.random_unitary(norb, seed=1234)

# Apply the orbital rotation to the statevector
vec = ffsim.apply_orbital_rotation(vec, orbital_rotation, norb=norb, nelec=nelec)

# Equivalent way to create the same state
n_alpha, n_beta = nelec
occupied_orbitals = (range(n_alpha), range(n_beta))
slater_det = ffsim.slater_determinant(
    norb, occupied_orbitals, orbital_rotation=orbital_rotation
)

np.testing.assert_allclose(slater_det, vec)

Here, `vec` is a plain one-dimensional NumPy array (a vector). Its length is determined by the number of orbitals and their occupancies. Because ffsim only implements transformations which conserve particle number and the Z component of spin, the number of $\alpha$-electrons and the number of $\beta$-electrons are each fixed. For a system with $N$ spatial orbitals, $N_\alpha$ electrons with spin $\alpha$, and $N_\beta$ electrons with spin $\beta$, the statevector has length

$$
{N \choose N_\alpha} \times {N \choose N_\beta}.
$$

In contrast, a generic quantum circuit simulator would represent the statevector in the space spanned by all possible bitstrings of length $2N$, resulting in a dimention of $2^{2N}$.

For convenience, ffsim includes functions to calculate these dimensions.

In [2]:
from scipy.special import comb

dim_a, dim_b = ffsim.dims(norb, nelec)
dim = ffsim.dim(norb, nelec)
print(f"The dimension of the vector space is {dim}.")
print(f"On the other hand, 2 ** (2 * norb) = {2 ** (2 * norb)}.")

assert dim_a == comb(norb, n_alpha, exact=True)
assert dim_b == comb(norb, n_beta, exact=True)
assert dim == dim_a * dim_b
assert vec.shape == (dim,)

The dimension of the vector space is 300.
On the other hand, 2 ** (2 * norb) = 4096.


This representation of the statevector is the same as that used in standard full configuration interaction (FCI) routines. It is often convenient to represent the statevector as a matrix whose rows are indexed by "$\alpha$-strings" describing the occupancies of the $\alpha$ orbitals, and columns indexed by "$\beta$-strings" describing the occupancies of the $\beta$ orbitals. To convert the vector into this representation, simply reshape it:

In [3]:
mat = vec.reshape((dim_a, dim_b))

The statevector representation depends on a choice of ordering for the $\alpha$- and $\beta$-strings. ffsim uses the same ordering as pySCF's FCI module, `pyscf.fci`. You can use the `indices_to_strings` function to convert a list of statevector indices to the corresponding bitstrings. The left half of a bitstring is the $\alpha$-string, and the right half is the $\beta$-string.

In [4]:
# Show the bitstrings for the first 20 indices
strings = ffsim.indices_to_strings(range(20), norb, nelec)
strings

['000111000011',
 '000111000101',
 '000111000110',
 '000111001001',
 '000111001010',
 '000111001100',
 '000111010001',
 '000111010010',
 '000111010100',
 '000111011000',
 '000111100001',
 '000111100010',
 '000111100100',
 '000111101000',
 '000111110000',
 '001011000011',
 '001011000101',
 '001011000110',
 '001011001001',
 '001011001010']